In [1]:
import tabula

# PDFファイルから表を読み込んでリストに格納
dfs = tabula.read_pdf("https://www.jtuc-rengo.or.jp/activity/roudou/shuntou/2023/yokyu_kaito/kaito/kaito_no7_ichiji.pdf?9711", pages="all")

# リストの中身を表示
for df in dfs:
    print(df)


JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.